# Bundled irreducible derivatives approach

Bundled irreducible derivatives approach (BID) tries to prove the entire system with as few displacements as possible. 
We compute the chainrule matrix for our displacement vectors after execute the symmetry analysis.
Once we find the displacement vectors that the rank of chainrule matrix match the number of 
irreducible derivatives we are looking to solve, we can use the set of displacements to compute the irreducible derivatives.

As an example, we atempt to compute the phonons of rocksalt with FTG of $2\hat{1}$, and we use DFT engine VASP to compute forces of displacements.

In [5]:
import numpy as np
from principia_materia.phonon_id.bid import BID
from principia_materia.io_interface.vasp import parse_poscar
from principia_materia.translation_group import get_structure, CrystalFTG

In [4]:
structure = get_structure(parse_poscar("nacl/POSCAR"), stype=CrystalFTG)
structure.orbitals = "p"

In [6]:
mesh = np.array([
  [2, 0, 0],
  [0, 2, 0],
  [0, 0, 2]])

In [7]:
bid = BID(structure=structure, supa=mesh, order=2, pg="Oh")
bid.set_chainrule_derivatives(verbose=True)
bid._set_displacement_generator()
bid.find_measurements(verbose=True)

Computing ADT for Qpoint: ((0, 0, 0), (0, 0, 0))
Computing ADT for Qpoint: ((1/2, 0, 0), (1/2, 0, 0))
Computing ADT for Qpoint: ((1/2, 1/2, 0), (1/2, 1/2, 0))
Summarize irreducible derivatives.
Compute vectorized tensors.
Number of unknowns:  11
Rank of single measurements:  11
Estimated number of measurements:  1


Now we have found the displacements needed to solve for the irreducible derivatives, we can create the dft jobs for certain DFT engine.

In [9]:
bid.create_dft_jobs(
    root_directory="test_bid",
    deltas=[0.01, 0.02, 0.03],
    dft_engine="vasp",
    jobname_prefix="test_job_",
    dry_run=True,
    )

In order to compute the finite displacements from the DFT results, 
perform quadratic error tail for the central finite difference, 
and solve for the irreducible derivatives, and compute dynamic tensors
of the FTG. 

```python
bid.get_fd_results(
    "test_bid",
    "forces",
    "vasp",
    config_file="finite_displacements.yml",
    )
bid.get_errortail_results()
bid.compute_irreducible_derivatives()
bid.get_dynamic_tensors()
```